# Instructions
Use this notebook to produce marked images with both model predictions and human labeling, only for images where there is a difference between the two. This can be used to find mistakes in human labeling as well as get a qualitative evaluation of the model (model fails to detect more than one of overlapping objects etc.). All labels should be in YOLO's txt format.
1. Run 
`python detect.py --weights your_model.pt --source path_to_images/ --save-txt --save-conf --nosave`
1. Put the path to the resulting model inferred `labels` folder in `path_a` 
1. Put the path to true labels in `path_b`
1. Put the path to images in `path_img`
1. Run this notebook
1. Optional: tweak model parameters and repeat **Detect Differences** stage until results match your needs

In [1]:
import os
import cv2
from tqdm.auto import tqdm

## Model Parameters

In [2]:
conf_tres = 0.5
iou_tres = 0.25
nms_tres = 0.5
cls_dict = {0: 'male', 1:'female'}

## Paths

In [3]:
path_a = 'conf/'
path_b = 'img/'
path_img = 'img/'
path_out = 'out/'

## Graphic Parameters

In [4]:
img_extension = 'jpg'
font = cv2.FONT_HERSHEY_SIMPLEX
font_size = 1.5
line_width = 2
lbl_color = (200, 0, 0)
pred_color = (0, 0, 200)

# Functions

In [5]:
def line2dict(line):
    str_list = line.split()
    inst = dict(
        cls = int(str_list[0]),
        x1 = float(str_list[1]) - 0.5*float(str_list[3]),
        y1 = float(str_list[2]) - 0.5*float(str_list[4]),
        x2 = float(str_list[1]) + 0.5*float(str_list[3]),
        y2 = float(str_list[2]) + 0.5*float(str_list[4]),
    )
    if len(str_list) > 5:
        inst['conf'] = float(str_list[5])
    return inst

In [6]:
def iou(inst_a, inst_b):
    # intersection width
    if inst_b['x1'] <= inst_a['x1'] <= inst_b['x2']:
        if inst_a['x2'] <= inst_b['x2']:
            intersection_w = inst_a['x2'] - inst_a['x1']
        else:
            intersection_w = inst_b['x2'] - inst_a['x1']
            
    elif inst_a['x1'] <= inst_b['x1'] <= inst_a['x2']:
        if inst_b['x2'] <= inst_a['x2']:
            intersection_w = inst_b['x2'] - inst_b['x1']
        else:
            intersection_w = inst_a['x2'] - inst_b['x1']
    
    else:
        intersection_w = 0
    
    # intersection height
    if inst_b['y1'] <= inst_a['y1'] <= inst_b['y2']:
        if inst_a['y2'] <= inst_b['y2']:
            intersection_h = inst_a['y2'] - inst_a['y1']
        else:
            intersection_h = inst_b['y2'] - inst_a['y1']
            
    elif inst_a['y1'] <= inst_b['y1'] <= inst_a['y2']:
        if inst_b['y2'] <= inst_a['y2']:
            intersection_h = inst_b['y2'] - inst_b['y1']
        else:
            intersection_h = inst_a['y2'] - inst_b['y1']
    
    else:
        intersection_h = 0
        
    # IoU
    intersection = intersection_w * intersection_h
    area_a = (inst_a['x2'] - inst_a['x1']) * (inst_a['y2'] - inst_a['y1'])
    area_b = (inst_b['x2'] - inst_b['x1']) * (inst_b['y2'] - inst_b['y1'])
    union = area_a + area_b - intersection
    iou = intersection / union
    
    return iou

In [7]:
def is_contained(inst_a, inst_b):
    ret = ((inst_b['x1'] <= inst_a['x1'] <= inst_b['x2'] and 
            inst_b['x1'] <= inst_a['x2'] <= inst_b['x2'] and
            inst_b['y1'] <= inst_a['y1'] <= inst_b['y2'] and
            inst_b['y1'] <= inst_a['y2'] <= inst_b['y2']) or
           (inst_a['x1'] <= inst_b['x1'] <= inst_a['x2'] and 
            inst_a['x1'] <= inst_b['x2'] <= inst_a['x2'] and
            inst_a['y1'] <= inst_b['y1'] <= inst_a['y2'] and
            inst_a['y1'] <= inst_b['y2'] <= inst_a['y2'])
          )
    return ret

In [8]:
def nms(pred_list, thres):
    keep = [pred_list[0]] if len(pred_list) else []
    for pred in pred_list[1:]:
        for kept in keep:
            if iou(kept, pred) > thres or is_contained(kept, pred):
                break
            keep.append(pred)
        
    return keep

In [9]:
def read_pred(path):
    if not os.path.exists(path):
        return []
    with open(path) as f:
        pred_list = nms([line2dict(line) for line in f if float(line.split()[-1]) >= conf_tres], nms_tres)
    return pred_list

In [10]:
def read_lbl(path):
    with open(path) as f:
        lbl_list = [line2dict(line) for line in f]
    return lbl_list

In [11]:
def mark_img(src, dst, pred_list, lbl_list):
    # mark prediction and label on same file
    img = cv2.imread(src)
    y_coeff = img.shape[0]
    x_coeff = img.shape[1]
    
    for lbl in lbl_list:
        cls = cls_dict[lbl['cls']]
        pos0 = (int(x_coeff*lbl['x1']), int(y_coeff*lbl['y1']))
        pos1 = (int(x_coeff*lbl['x2']), int(y_coeff*lbl['y2']))
        cv2.rectangle(img, pos0, pos1, lbl_color ,line_width)
        cv2.putText(img, cls, (pos0[0], pos0[1]-int(5*font_size)), font, font_size, lbl_color, 3)
        
    for pred in pred_list:
        cls = cls_dict[pred['cls']]
        conf = pred['conf']
        desc = ' '.join([cls, str(round(conf, 3))])
        pos0 = (int(x_coeff*pred['x1']), int(y_coeff*pred['y1']))
        pos1 = (int(x_coeff*pred['x2']), int(y_coeff*pred['y2']))
        cv2.rectangle(img, pos0, pos1, pred_color ,line_width)
        cv2.putText(img, desc, (pos0[0], pos1[1]+int(20*font_size)), font, font_size, pred_color, 3)
        
    cv2.imwrite(dst, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

## Detect Differences

In [12]:
txt_list = [f for f in os.listdir(path_b) if f[-4:] == '.txt']
mislabels = []
misses = []
false_positives = []
counters = {value: 0 for value in cls_dict.values()}
log = ''

for txt in tqdm(txt_list, desc='comparing'):
    pred_list = read_pred(os.path.join(path_a, txt))
    lbl_list = read_lbl(os.path.join(path_b, txt))
    
    for lbl in lbl_list:
        hit = False
        counters[cls_dict[lbl['cls']]] += 1
        for pred in pred_list:
            if iou(lbl, pred) >= iou_tres or is_contained(lbl, pred):
                hit = True
                if pred['cls'] != lbl['cls']:
                    log += ' '.join([txt, cls_dict[lbl['cls']], 'mislabel']) + '\n'
                    mislabels.append(txt)
                break
        if not hit:
            log += ' '.join([txt, cls_dict[lbl['cls']], 'miss']) + '\n'
            misses.append(txt)

    for pred in pred_list:
        hit = False
        for lbl in lbl_list:
            if iou(lbl, pred) >= iou_tres or is_contained(lbl, pred):
                hit = True
                break
        if not hit:
            log += ' '.join([txt, cls_dict[lbl['cls']], 'false positive']) + '\n'
            false_positives.append(txt)

comparing:   0%|          | 0/365 [00:00<?, ?it/s]

In [13]:
print('Label counts:', counters)

Label counts: {'male': 251, 'female': 195}


In [14]:
to_mark = [*set(misses+mislabels+false_positives)]
print('miss lbl  FP   files')
print('{:<5d}{:<5d}{:<5d}{:<5d}'.format(len(misses), len(mislabels), len(false_positives), len(to_mark)))

miss lbl  FP   files
0    5    3    8    


In [15]:
if not os.path.isdir(path_out):
    os.mkdir(path_out)

In [16]:
with open(os.path.join(path_out, 'log.txt'), 'w') as f:
    f.write(log)
    f.write('\nmiss lbl  FP   files\n')
    f.write('{:<5d}{:<5d}{:<5d}{:<5d}\n'.format(len(misses), len(mislabels), len(false_positives), len(to_mark)))
    f.write('\nLabel counts: ' + str(counters))

# Mark Images with Differences

In [17]:
for txt in tqdm(to_mark, desc='visualizing'):
    pred_list = read_pred(os.path.join(path_a, txt))
    lbl_list = read_lbl(os.path.join(path_b, txt))
    img_filename = txt[:-3] + img_extension
    src = os.path.join(path_img, img_filename)
    dst = os.path.join(path_out, img_filename)
    mark_img(src, dst, pred_list, lbl_list)

visualizing:   0%|          | 0/8 [00:00<?, ?it/s]